In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

from ionpy.analysis import ResultsLoader
from ese.experiment.experiment import CalibrationExperiment
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '1' 

%load_ext yamlmagic
%load_ext autoreload

%autoreload 2

In [ ]:
# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path = f"{root}/WMH_aug_runs"

dfc = rs.load_configs(
    path,
    properties=False,
)

df = rs.load_metrics(dfc)

In [ ]:
best_exp = rs.get_experiment(
    df=df,
    exp_class=CalibrationExperiment,
    metric="val-dice_score",
    checkpoint="max-val-dice_score",
    device="cuda"
)

In [ ]:
best_exp.vis_loss_curves(height=6)

In [ ]:
%%yaml dataset_cfg 

_class: ese.experiment.datasets.WMH
annotator: observer_o12
axis: 0
split: cal 
num_slices: 1
slicing: midslice 
task: Amsterdam 
version: 0.2

In [ ]:
from ionpy.experiment.util import absolute_import
from torch.utils.data import DataLoader

dataset_cls = absolute_import(dataset_cfg.pop("_class"))
WMH_Dataset = dataset_cls(**dataset_cfg)
wmh_dataloader = DataLoader(WMH_Dataset, batch_size=1, shuffle=False, drop_last=False)

In [ ]:
import torch
import einops
import numpy as np
from tqdm.notebook import tqdm
from ese.experiment.metrics.grouping.portion_loss import get_perpix_boundary_dist, get_perpix_group_size
from ese.experiment.metrics.grouping.regions import get_label_region_sizes
from ionpy.util.torchutils import to_device

def get_dataset_perf(
        exp, 
        dataloader, 
        ):
    items = []
    with torch.no_grad():
        for subj_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
            # Get your image label pair and define some regions.
            x, y = to_device(batch, exp.device)
            # Reshape to a good size
            x = einops.rearrange(x, "b c h w -> (b c) 1 h w")
            y = einops.rearrange(y, "b c h w -> (b c) 1 h w")
            # Get the prediction
            yhat = exp.model(x)  
            # Extract predictions
            soft_pred = torch.sigmoid(yhat)
            # Get the hard prediction
            hard_pred = (soft_pred > 0.5).float()
            # Squeeze our tensors
            x = x.squeeze().cpu().numpy()
            y = y.squeeze().cpu().numpy()
            soft_pred = soft_pred.squeeze().cpu().numpy()
            hard_pred = hard_pred.squeeze().cpu().numpy()
            # Get some performance metrics
            accuracy_map = (hard_pred==y).astype(np.float32)
            perf_per_dist = get_perpix_boundary_dist(y_pred=hard_pred)
            perf_per_regsize = get_perpix_group_size(y_pred=hard_pred)
            # Get region sizes
            gt_lab_region_sizes = get_label_region_sizes(label_map=y)
            pred_lab_region_sizes = get_label_region_sizes(label_map=hard_pred)
            # Wrap it in an item
            items.append({
                "subject_id": subj_idx,
                "image": x,
                "label_map": y,
                "conf_map": soft_pred,
                "pred_map": hard_pred,
                "perpix_accuracies": accuracy_map,
                "perf_per_dist": perf_per_dist,
                "perf_per_regsize": perf_per_regsize,
                "gt_lab_region_sizes": gt_lab_region_sizes,
                "pred_lab_region_sizes": pred_lab_region_sizes
            })
    return items

In [ ]:
# val_perf is a dict where each item is the subj id
# with the y, ypred, yloss, ydice
predictions_list = get_dataset_perf(
    exp=best_exp, 
    dataloader=wmh_dataloader
    )

In [ ]:
from ese.experiment.analysis.inference import get_pixelinfo_df

pixel_preds_df = get_pixelinfo_df(predictions_list)

In [ ]:
len(pixel_preds_df)

## Distribution of Pixel Accuracies

In [ ]:
sns.barplot(data=pixel_preds_df, y="accuracy")

## Distribution of Pixel Accuracies vs Label 

In [ ]:
sns.barplot(data=pixel_preds_df, x="label", y="accuracy")


## Distribution Predicted Region Sizes Over Images. 

In [ ]:
from ese.experiment.analysis.err_diagrams import viz_region_size_distribution

viz_region_size_distribution(
    data_points=predictions_list,
    hue="label_map_type"
)

In [ ]:
from ese.experiment.analysis.err_diagrams import viz_region_size_distribution

viz_region_size_distribution(
    data_points=predictions_list,
    hue="label_map_type",
    row_split="label"
)

## Distribution of Pixel Accuracies vs Distance to a Boundary

In [ ]:
sns.lineplot(data=pixel_preds_df, x="dist_to_boundary", y="accuracy")

In [ ]:
unique_labels = pixel_preds_df['label'].unique()
color_dict = {
    0.0: "blue",
    1.0: "orange"
}
for l_idx, lab in enumerate(unique_labels):
    sns.lineplot(
        data=pixel_preds_df[pixel_preds_df['label']==lab], 
        x="dist_to_boundary", 
        y="accuracy",
        color=sns.color_palette("tab10")[l_idx]
        )
    plt.title(f"Label {lab}")
    plt.show()

## Distribution Pixel Accuracies vs Predicted Size of Region (that pixel is in)

In [ ]:
sns.lineplot(data=pixel_preds_df, x="group_size", y="accuracy")

In [ ]:
unique_labels = pixel_preds_df['label'].unique()
for l_idx, lab in enumerate(unique_labels):
    sns.lineplot(
        data=pixel_preds_df[pixel_preds_df['label']==lab],
        x="group_size", 
        y="accuracy",
        color=sns.color_palette("tab10")[l_idx]
        )
    plt.title(f"Label {lab}")
    plt.show()